# GAMES ANALYSIS PROJECT

# Setup

In [23]:
# importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [24]:
# loading the datasets.
# i have already downloaded the datasets from the following links:

# https://www.kaggle.com/datasets/kapturovalexander/ign-games-from-best-to-worst/data
games_data = pd.read_csv(r'C:\!Projects\games_analysis\Data\IGN_games_ratings.csv')
games = games_data.copy()

# https://www.kaggle.com/datasets/hibrahimag1/top-1000-twitch-streamers-data-may-2024
twitch_streamers = pd.read_csv(r'C:\!Projects\games_analysis\Data\Twitch_streamers_data.csv')
streamers = twitch_streamers.copy()

# https://www.kaggle.com/datasets/rankirsh/evolution-of-top-games-on-twitch
twitch_stat = pd.read_csv(r'C:\!Projects\games_analysis\Data\twitch_games_data.csv', encoding='latin1')
twitch = twitch_stat.copy()

# EDA

## Quick view of the datasets

In [25]:
games.head()

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
0,Checkered Flag,10.0,Masterpiece,Lynx,Racing,1999,7,6
1,Chrono Trigger,10.0,Masterpiece,Wii,"Action, RPG",2011,5,25
2,Dragon Warrior III,10.0,Masterpiece,Game Boy Color,RPG,2001,7,20
3,Grand Theft Auto IV,10.0,Masterpiece,Xbox 360,"Action, Adventure",2008,4,25
4,Grand Theft Auto IV,10.0,Masterpiece,PlayStation 3,"Action, Adventure",2008,4,25


In [26]:
streamers.head()

,RANK,NAME,LANGUAGE,TYPE,MOST_STREAMED_GAME,2ND_MOST_STREAMED_GAME,AVERAGE_STREAM_DURATION,FOLLOWERS_GAINED_PER_STREAM,AVG_VIEWERS_PER_STREAM,AVG_GAMES_PER_STREAM,TOTAL_TIME_STREAMED,TOTAL_FOLLOWERS,TOTAL_VIEWS,TOTAL_GAMES_STREAMED,ACTIVE_DAYS_PER_WEEK,MOST_ACTIVE_DAY,DAY_WITH_MOST_FOLLOWERS_GAINED
0,1,kaicenat,English,personality,Just Chatting,I'm Only Sleeping,7.6,18.405,15.852,2.3,4.698,10600000,9150000,194,3.6,Friday,Saturday
1,2,jynxzi,English,personality,Tom Clancy's Rainbow Six Siege,NBA 2K20,5.4,3.386,1.145,1.2,8.407,5760000,1950000,54,5.6,Tuesday,Sunday
2,3,caedrel,English,personality,League of Legends,I'm Only Sleeping,6.3,689.000,12.331,1.3,6.728,797000,14200000,111,2.8,Thursday,Sunday
3,4,caseoh_,English,personality,NBA 2K23,Just Chatting,4.6,7.185,0.000,3.6,2.554,4220000,53,385,6.2,Friday,Monday
4,5,ibai,Spanish,personality,Just Chatting,League of Legends,4.1,8.289,190.714,1.5,6.865,15600000,359000000,149,4.3,Wednesday,Saturday


In [27]:
twitch.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
0,1,League of Legends,1,2016,94377226,1362044,530270,2903,129172,127021,1833,69.29
1,2,Counter-Strike: Global Offensive,1,2016,47832863,830105,372654,2197,120849,64378,1117,57.62
2,3,Dota 2,1,2016,45185893,433397,315083,1100,44074,60815,583,104.26
3,4,Hearthstone,1,2016,39936159,235903,131357,517,36170,53749,317,169.29
4,5,Call of Duty: Black Ops III,1,2016,16153057,1151578,71639,3620,214054,21740,1549,14.03


## Checking missings and duplicated rows

In [28]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18625 entries, 0 to 18624
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          18625 non-null  object 
 1   score          18625 non-null  float64
 2   score_phrase   18625 non-null  object 
 3   platform       18625 non-null  object 
 4   genre          18589 non-null  object 
 5   release_year   18625 non-null  int64  
 6   release_month  18625 non-null  int64  
 7   release_day    18625 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 1.1+ MB


In [34]:
round(games.isnull().mean()*100,2)

title            0.00
score            0.00
score_phrase     0.00
platform         0.00
genre            0.19
release_year     0.00
release_month    0.00
release_day      0.00
dtype: float64

In [35]:
games.genre.unique()

array(['Racing', 'Action, RPG', 'RPG', 'Action, Adventure', 'Action',
       'Fighting, Action', 'Adventure', 'Puzzle, Adventure',
       'Sports, Action', 'Fighting', 'Puzzle', 'Platformer', 'Strategy',
       'Sports', 'Racing, Simulation', 'Shooter', 'Compilation',
       'Racing, Action', 'Simulation', 'Strategy, RPG', 'Music',
       'Puzzle, Action', 'Party', 'Music, Action', 'Sports, Simulation',
       'Compilation, Compilation', 'Card', 'Flight, Action', 'Hardware',
       'Shooter, RPG', 'Action, Simulation', 'Wrestling',
       'Platformer, Action', 'Puzzle, RPG', 'Trivia', 'Board',
       'Productivity', nan, 'Action, Compilation', 'Card, Battle',
       'Music, Editor', 'Sports, Racing', 'Flight, Simulation', 'Other',
       'Platformer, Adventure', 'Adventure, Compilation', 'Other, Action',
       'Flight', 'Card, RPG', 'Pinball', 'Educational, Action',
       'RPG, Simulation', 'Adventure, Episodic', 'Shooter, First-Person',
       'Music, Adventure', 'Casino', 'Virtual 

In [37]:
games.genre.fillna('Not known', inplace=True)

In [39]:
games.isnull().sum()

title            0
score            0
score_phrase     0
platform         0
genre            0
release_year     0
release_month    0
release_day      0
dtype: int64

# AI

In [ ]:
# Using Gemini OpenAI
import google.generativeai as genai
import apikey

key = apikey.load("game_api")
genai.configure(api_key=key)

model = genai.GenerativeModel('gemini-1.5-flash')
# response = model.generate_content("Write a story about a magic backpack.")
# print(response.text)